# Lab 2 Part 1: PRISM & Spacetime Cube

In [9]:
import arcpy
import requests
import os
import numpy as np
import zipfile

In [11]:
#define and call url
prism_url = 'https://www.prism.oregonstate.edu/fetchData.php?type=all_bil&kind=normals&spatial=4km&elem=ppt&temporal=annual'
#set path
prismpath = 'C:/Users/schil726/Downloads/GIS5571_Lab2/prism/precip_30y_normals.zip'
#make call
prism_response = requests.get(prism_url)
#save locally
with open(prismpath, 'wb') as file:
    file.write(prism_response.content)
print("downloaded prism zip")

downloaded prism zip


In [12]:
#extract the prism zip
with zipfile.ZipFile(prismpath, 'r') as zip_ref:
    zip_ref.extractall('C:/Users/schil726/Downloads/GIS5571_Lab2/prism/BIL/')

In [18]:
#create empty mosaic dataset then add all BIL files
arcpy.management.CreateMosaicDataset(
    in_workspace = "C:/Users/schil726/Documents/ArcGIS/Projects/GIS5571_Lab2/GIS5571_Lab2.gdb",
    in_mosaicdataset_name = "PRISM_mosaic",
    coordinate_system = arcpy.SpatialReference(102003)
)

<Result 'C:\\Users\\schil726\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2\\GIS5571_Lab2.gdb\\PRISM_mosaic'>

In [19]:
#add BIL files to dataset
arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset = "C:/Users/schil726/Documents/ArcGIS/Projects/GIS5571_Lab2/GIS5571_Lab2.gdb/PRISM_mosaic",
    raster_type = "Raster Dataset",
    input_path = "C:/Users/schil726/Downloads/GIS5571_Lab2/prism/BIL",
    sub_folder = "SUBFOLDERS",
    duplicate_items_action = "EXCLUDE_DUPLICATES",
    build_pyramids = "BUILD_PYRAMIDS",
    calculate_statistics = "CALCULATE_STATISTICS",
    force_spatial_reference = "NO_FORCE_SPATIAL_REFERENCE",
    estimate_statistics = "ESTIMATE_STATISTICS",
)

<Result 'C:/Users/schil726/Documents/ArcGIS/Projects/GIS5571_Lab2/GIS5571_Lab2.gdb/PRISM_mosaic'>

In [25]:
#calculate variable field
arcpy.management.CalculateField(
    in_table = "PRISM_mosaic/Footprint",
    field = "Variable",
    expression = '"Precipitation"',
    code_block = "",
    expression_type = "PYTHON3",
    field_type = "TEXT",
)

<Result 'PRISM_mosaic\\Footprint'>

In [29]:
#calculate timestamp field
arcpy.management.CalculateField(
    in_table = "PRISM_mosaic/Footprint",
    field = "Timestamp", 
    expression = "DateAdd(Date(2024, 0, 1), $feature.OBJECTID-4, 'month')",
    expression_type = "ARCADE",
    code_block = "", 
    field_type = "DATE",
)

<Result 'PRISM_mosaic\\Footprint'>

In [31]:
#build multidimensional info
arcpy.management.BuildMultidimensionalInfo(
    in_mosaic_dataset = "PRISM_mosaic",
    variable_field = "Variable",
    dimension_fields = "Timestamp",
)

<Result 'PRISM_mosaic'>

In [32]:
#make multidimensional raster layer
arcpy.md.MakeMultidimensionalRasterLayer(
    in_multidimensional_raster = "PRISM_mosaic",
    out_multidimensional_raster_layer = "Multidimensional_PRISM",
    template = "DISPLAY",
)

<Result 'Multidimensional_PRISM'>

In [33]:
#and now, finally, create spacetime cubes!
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(
    in_md_raster = "Multidimensional_PRISM",
    output_cube = "C:/Users/schil726/Downloads/GIS5571_Lab2/prism/PRISM_stcubes.nc",
    fill_empty_bins = "SPATIAL_NEIGHBORS",
)

<Result 'C:\\Users\\schil726\\Downloads\\GIS5571_Lab2\\prism\\PRISM_stcubes.nc'>